In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc
from datetime import datetime

In [ ]:
# We start by importing the data via Excel, and checking the types of our columns before doing anything else!
df = pd.read_excel("TSLA SP500 Datas.xlsx", sheet_name = 'TSLA')
df.dtypes


In [ ]:
df["S&P 500 Adj Close"] = pd.to_numeric(df["S&P 500 Adj Close"])  
#df["S&P 500 Volume"] = pd.to_numeric(df["S&P 500 Volume"])  

In [ ]:
st_Name = 'TESLA'
index_Name = 'S&P 500'
st_Ticker = 'TSLA'
event_Type = 'Stock Split'

#Event date = 31-08-2020
year = 2020
month = 8
day = 31

Index_event = df.loc[(df["Date"].dt.year ==2020)&(df["Date"].dt.month==8)&(df["Date"].dt.day==31)].index[0]
df_EventWindow = df.loc[Index_event : Index_event + 5, :]
df_ControlWindow = df.loc[Index_event-120 : Index_event-6, :]

df = pd.concat([df_ControlWindow, df_EventWindow], axis = 0)
df.reset_index(drop = True, inplace = True)

st_Return = df.iloc[-1,1] / df.iloc[0,1] - 1 

print(df)
print('==========================================')
print('              KEY INFORMATION             ')
print('==========================================')
print('Corporation =', st_Name)
print('Ticker =', st_Ticker)
print('Event Nature = ', event_Type)
print('Event Date = ', day, '/',month, '/',year)
print('Dates = from', (df.iloc[0,0]).strftime('%d/%m/%Y'),'to',  (df.iloc[-1,0]).strftime('%d/%m/%Y'))
print('Stock Return over the period  = ', str(round(st_Return * 100, 3)), '%')
print('==========================================')


In [ ]:
def Plot_Price_Volume (Date,Volume,Price,df) : 
    
    x=df[Date]
    y=df[Volume]
    z=df[Price]
    
    fig, ax1 = plt.subplots(figsize=(12,6))
    color = 'blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('volume', color=color)
    ax1.bar(x, y, label=Volume)
    ax1.tick_params(axis='x', labelcolor='grey')
    #ax1.legend(loc="upper left")
    #plt.legend()
    plt.legend(loc="upper left")
    

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'red'
    ax2.set_ylabel('adj price', color=color)  # we already handled the x-label with ax1
    ax2.plot(x, z, color=color, label=Price)
    ax2.tick_params(axis='x', labelcolor=color)
    ax2.legend(loc=(0.006,0.89))
    #plt.legend(loc=(0.006,0.89))

        
    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    return plt.show()

Plot_Price_Volume("Date",'TESLA Volume', 'TSLA Adj Close', df)
#the stock volume is very low near the split period, trading temporarily halted?

In [ ]:
df_ControlWindow['Tesla Return'] = (df_ControlWindow["TSLA Adj Close"] - df_ControlWindow.shift(periods = 1)["TSLA Adj Close"]) / df_ControlWindow.shift(periods = 1)["TSLA Adj Close"]
df_ControlWindow['Index Return'] = (df_ControlWindow["S&P 500 Adj Close"] - df_ControlWindow.shift(periods = 1)["S&P 500 Adj Close"]) /df_ControlWindow.shift(periods = 1)["S&P 500 Adj Close"]
#df_ControlWindow

df_EventWindow['Tesla Return'] = (df_EventWindow["TSLA Adj Close"] - df_EventWindow.shift(periods = 1)["TSLA Adj Close"]) / df_EventWindow.shift(periods = 1)["TSLA Adj Close"]
df_EventWindow['Index Return'] = (df_EventWindow["S&P 500 Adj Close"] - df_EventWindow.shift(periods = 1)["S&P 500 Adj Close"]) / df_EventWindow.shift(periods = 1)["S&P 500 Adj Close"]
#df_EventWindow

df_Returns = pd.concat([df_ControlWindow, df_EventWindow])
df_Returns.reset_index(drop = True, inplace = True)
#df_Returns
df_Returns[110:]  #just to make sure that there is no problem in the concatenation

In [ ]:
# Average Returns
st_CW_Mean = df_ControlWindow['Tesla Return'].mean()
st_EW_Mean = df_EventWindow['Tesla Return'].mean()
index_CW_Mean = df_ControlWindow['Index Return'].mean()
index_EW_Mean = df_EventWindow['Index Return'].mean()


# Volatility of Returns
st_CW_VolatilityRt = df_ControlWindow['Tesla Return'].std()
index_CW_VolatilityRt = df_ControlWindow['Index Return'].std()
st_EW_VolatilityRt = df_EventWindow['Tesla Return'].std()
index_EW_VolatilityRt = df_EventWindow['Index Return'].std()

print('==========================================')
print('              CONTROL WINDOW              ')
print('==========================================')
print('Dates = from', (df_ControlWindow.iloc[0,0]).strftime('%d/%m/%Y'),'to',  (df_ControlWindow.iloc[-1,0]).strftime('%d/%m/%Y'))
print(st_Name, 'Average Return =', str(round(st_CW_Mean * 100, 4)),'%')
print(st_Name, 'Return Volatility =', str(round(st_CW_VolatilityRt * 100, 4)),'%')
print(index_Name, 'Average Return =', str(round(st_CW_Mean * 100, 4)),'%')
print(index_Name, 'Return Volatility =', str(round(index_CW_VolatilityRt * 100, 4)),'%')
print('==========================================')
print('')
print('')
print('==========================================')
print('              EVENT WINDOW              ')
print('==========================================')
print('Dates = from', (df_EventWindow.iloc[0,0]).strftime('%d/%m/%Y'),'to',  (df_EventWindow.iloc[-1,0]).strftime('%d/%m/%Y'))
print(st_Name, 'Average Return =', str(round(st_EW_Mean * 100, 4)),'%')
print(st_Name, 'Return Volatility =', str(round(st_EW_VolatilityRt * 100, 4)),'%')
print(index_Name, 'Average Return =', str(round(index_EW_Mean * 100, 4)),'%')
print(index_Name, 'Return Volatility =', str(round(index_EW_VolatilityRt * 100, 4)),'%')
print('==========================================')

In [ ]:
def Draw_Linear_Regression(x,y,DataFrame) : 
    X_label=x
    Y_label=y
    
    x=np.asarray(DataFrame[x])
    y=np.asarray(DataFrame[y])
    
    line=sc.linregress(x,y)
    line_slope=line.slope
    line_intercept=line.intercept
    line_correl=line.rvalue
       
    My_Modele=line_slope*x + line_intercept
    
    plt.figure(figsize=(10, 4))
    plt.scatter(x,y,label='observed data')
    plt.plot(x,My_Modele, c='r',label='fitted data')    
    
    plt.xlabel(X_label)
    plt.ylabel(Y_label)
    
    plt.title("Linear Regression")
    plt.legend()
    plt.show()
 
    return (plt.show(),"The estimated market model is : " + "Y = " + str(line[0]) + " X + "+ str(round(line_intercept,7)))[1]
    
Draw_Linear_Regression('Index Return','Tesla Return', df_ControlWindow[1:])

# We tried to fill the nan with mean value, but it is not mandatory :
#df_ControlWindow['Tesla Return'] = (df_ControlWindow["TSLA Adj Close"] - df_ControlWindow.shift(periods = 1)["TSLA Adj Close"]) / df_ControlWindow.shift(periods = 1)["TSLA Adj Close"]
#df_ControlWindow["Tesla Return"].fillna(np.mean(df_ControlWindow["Tesla Return"]),inplace=True)
#df_ControlWindow["Index Return"].fillna(np.mean(df_ControlWindow["Index Return"]),inplace=True)
#Linear_Regression('Index Return','Tesla Return', df_ControlWindow)

In [ ]:
def Linear_Regression(x,y,DataFrame) : 
    X_label=x
    Y_label=y
    x=np.asarray(DataFrame[x])
    y=np.asarray(DataFrame[y])
    line=sc.linregress(x,y)
    return line

slope = Linear_Regression('Index Return','Tesla Return', df_ControlWindow[1:])[0]
intercept = Linear_Regression('Index Return','Tesla Return', df_ControlWindow[1:])[1]

df_ControlWindow['Abnormal Return'] = df_ControlWindow['Tesla Return'] - (slope * df_ControlWindow['Index Return'] - intercept)
df_EventWindow['Abnormal Return'] = df_EventWindow['Tesla Return'] - (slope * df_EventWindow['Index Return'] - intercept)

In [ ]:
df_EventWindow

In [ ]:
df_ControlWindow

In [ ]:
#Q14
CAR_EW = df_EventWindow['Abnormal Return'].sum()
CAR_EW

In [ ]:
#Q15 Statistic Test
AR_CW_Volatility = df_ControlWindow['Abnormal Return'].std()
Stat = CAR_EW / (np.sqrt(6) * AR_CW_Volatility)
Stat